In [1]:
from pyomo.environ import *

In [56]:
# instanciar o modelo
model = AbstractModel()

# parametros
model.n = Param(initialize = 4)
model.i = Param(initialize = 4)
model.j = Param(initialize = 4)
model.c = Param(initialize = 2) # classes
model.CT = Param(initialize = 10) # capacidade do trem

precos_ini = {(1,2,1):20,(1,3,1):45,(1,4,1):75,(2,3,1):25,(2,4,1):55,(3,4,1):30,(1,2,2):25,(1,3,2):69,(1,4,2):88,(2,3,2):32,(2,4,2):93,(3,4,2):33}

precos = {(i,j,c): precos_ini[(i,j,c)] if (i,j,c) in precos_ini else 0  for i in range(1, 4+1) for j in range(1, 4+1) for c in range(1, 2+1) }


# conjuntos
model.I = RangeSet(1, model.i)
model.J = RangeSet(1, model.j)
model.C = RangeSet(1, model.c)
model.X0 = RangeSet(0, model.i)

#parametros indexados
model.COS = Param(model.I, model.J, model.C, initialize = precos)

# variaveis de desicao
model.X = Var(model.X0, model.J, model.C, within = Integers)
model.CAP = Var(model.X0, within = Integers)

# funcao objetivo
def fo(model):
    return sum( model.COS[i,j,c]*model.X[i,j,c] for i in model.I for j in model.J for c in model.C)
model.OBJ = Objective(rule = fo, sense = maximize)

# restricoes
def capacidadeP1(model, i):
    if i < model.n:
        return model.CAP[i] == model.CAP[i-1] + sum(model.X[i,j,c] for j in model.J for c in model.C if j>i) - sum(model.X[i-1,i,c] for c in model.C)
    else:
        return Constraint.Skip
model.ConstrainCapacidadeP1 = Constraint(model.I, rule = capacidadeP1)

def capacidadeP2(model, i):
    if i < model.n:
        return model.CAP[i] <= model.CT
    else:
        return Constraint.Skip
model.ConstrainCapacidadeP2 = Constraint(model.I, rule=capacidadeP2)


def iniCap0(model):
    return model.CAP[0] == 0
model.ConstrainIniCap0 = Constraint(rule=iniCap0)

def iniY0(model, c):
    return model.X[0,1,c] == 0
model.ConstraininiY0 = Constraint(model.C, rule=iniY0)

instance = model.create_instance()

opt = SolverFactory('cplex')

results = opt.solve(instance, tee=True)

# instance.pprint()
print(results)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\Wilmer\AppData\Local\Temp\tmpl2wqbuez.cplex.log' open.
CPLEX> Problem 'C:\Users\Wilmer\AppData\Local\Temp\tmpc6ow2zlp.pyomo.lp' read.
Read time = 0.00 sec. (0.00 ticks)
CPLEX> Problem name         : C:\Users\Wilmer\AppData\Local\Temp\tmpc6ow2zlp.pyomo.lp
Objective sense      : Maximize
Variables            :      18  [General Integer: 18]
Objective nonzeros   :      12
Linear constraints   :       9  [Less: 3,  Equal: 6]
  Nonzeros           :      30
  RHS nonzeros       :       3

Variables            : Min LB: all infinite     Max UB: all infinite   
Objective nonzeros   : Min   : 

In [61]:
print(value(instance.X[1,2,1]))

ERROR: evaluating object as numeric value: X[1,2,1]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object X[1,2,1]


ValueError: No value for uninitialized NumericValue object X[1,2,1]